In [1]:
import yaml
import cv2
assert cv2.__version__[0] >= '3', 'The fisheye module requires opencv version >= 3.1.1'
import numpy as np
import glob
import os

Calibration of the camera using images of the calibration pattern

In [2]:
CHECKERBOARD = (5,8)
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
_img_shape = None
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob(r"C:\Repo\GEOMAPI\tests\Kalibrationsamples\kal\*.jpg")


for fname in images:
    img = cv2.imread(fname)
    if _img_shape == None:
        _img_shape = img.shape[:2]
    else:
        assert _img_shape == img.shape[:2], "All images must share the same size."
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
DIM = gray.shape[::-1]
N_OK = len(objpoints)
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
rms, _, _, _, _ = \
    cv2.fisheye.calibrate(
        objpoints,
        imgpoints,
        gray.shape[::-1],
        K,
        D,
        rvecs,
        tvecs,
        calibration_flags,
        (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )
print("Found " + str(N_OK) + " valid images for calibration")
print("DIM=" + str(_img_shape[::-1]))
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")

Found 41 valid images for calibration
DIM=(3648, 5472)
K=np.array([[1403.442016286783, 0.0, 1840.6343029057366], [0.0, 1404.3440637808894, 2750.572088406907], [0.0, 0.0, 1.0]])
D=np.array([[0.00818846874508137], [0.0008806729094210791], [-0.0008884260133839184], [0.0002617989606467281]])


Undistort the iamges taken by the fish eyes

In [3]:

def undistort(img_path, balance=0.0, dim2=None, dim3=None):
    img = cv2.imread(img_path)

    dim1 = img.shape[:2][::-1]  #dim1 is the dimension of input image to un-distort
    assert dim1[0]/dim1[1] == DIM[0]/DIM[1], "Image to undistort needs to have same aspect ratio as the ones used in calibration"
    if not dim2:
        dim2 = dim1
    if not dim3:
        dim3 = dim1
    scaled_K = K * dim1[0] / DIM[0]  # The values of K is to scale with image dimension.
    scaled_K[2][2] = 1.0  # Except that K[2][2] is always 1.0
    # This is how scaled_K, dim2 and balance are used to determine the final K used to un-distort image. OpenCV document failed to make this clear!
    new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(scaled_K, D, dim2, np.eye(3), balance=balance)
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(scaled_K, D, np.eye(3), new_K, dim3, cv2.CV_16SC2)
    undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

    return undistorted_img
        

In [4]:

raw_dir = glob.glob(r"K:\Projects\2025-02 Project BAEKELAND MEETHET\7.Recordings\2021-02-NAV-Technologycampus_Ghent\DATA\VLX\datasets_proc\2021-02-09_10.39.09\cam\*.jpg")
undistorted_dir = r"C:\Repo\GEOMAPI\tests\Kalibrationsamples\Results\2021-02-09_10.39.09"

tags = []
for fname in raw_dir:
    filename = fname.split('\\')[-1]
    tag = filename.split("-")[0]
    if tag not in tags:
        tags.append(tag)

    undistorted_img = undistort(fname, balance=0.0)
    cv2.imwrite(os.path.join(undistorted_dir,filename), undistorted_img)
    
